# Part 3: Softmax Regression

In [0]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

In the second part of the lab we saw how to make a linear binary classifier using logisitic regression. In this  part of the lab we'll turn our attention to multi-class classification.

Softmax regression (or multinomial logistic regression) is a generalisation of logistic regression to the case where we want to handle multiple classes. In logistic regression we assumed that the labels were binary: $y_i\in \{0,1\}$. We used such a classifier to distinguish between two kinds of hand-written digits. Softmax regression allows us to handle $y_i \in \{1,\dots,K\}$ where $K$ is the number of classes.

Recall that in logistic regression, we had a training set $\{(\mathbf{x}_1,y_1),\dots,(\mathbf{x}_m,y_m)\}$ of $m$ labeled examples, where the input features are $\mathbf{x}_i \in \mathbb{R}^n$. In logistic regression, our hypothesis took the form:

\begin{align}
h_\theta(\mathbf{x}) &= \frac{1}{1 + \exp(-\mathbf{x}^\top\theta)} \equiv \sigma(\mathbf{x}^\top\theta)
\end{align}

and the model parameters $\theta$ were trained to minimise the cost function

\begin{align}
J(\theta) & = \sum_i y_i \log(\sigma(\mathbf{x}_i^\top\theta)) + (1-y_i) \log(1-\sigma(\mathbf{x}_i^\top\theta))
\end{align}

In the softmax regression setting, we are interested in multi-class classification, and so the label $y$
 can take on $K$ different values, rather than only two. Thus, in our training set $\{(\mathbf{x}_1,y_1),\dots,(\mathbf{x}_m,y_m)\}$, we now have that $y_i \in \{1,\dots,K\}$.

Given a test input $\mathbf{x}$, we want our hypothesis to estimate the probability that $P(y=k|\mathbf{x})$ for each value of $k=1,\dots,K$. That is to say, we want to estimate the probability of the class label taking on each of the $K$ different possible values. Thus, our hypothesis will output a $K$-dimensional vector (whose elements sum to 1) giving us our $K$ estimated probabilities. Concretely, our hypothesis $h_\theta(\mathbf{x})$ takes the form:

\begin{align}
h_\theta(\mathbf{x}) =
\begin{bmatrix}
P(y = 1 | \mathbf{x}; \theta) \\
P(y = 2 | \mathbf{x}; \theta) \\
\vdots \\
P(y = K | \mathbf{x}; \theta)
\end{bmatrix}
=
\frac{1}{ \sum_{j=1}^{K}{\exp(\theta^{(j)\top} \mathbf{x}) }}
\begin{bmatrix}
\exp(\theta^{(1)\top} \mathbf{x} ) \\
\exp(\theta^{(2)\top} \mathbf{x} ) \\
\vdots \\
\exp(\theta^{(K)\top} \mathbf{x} ) \\
\end{bmatrix}
\end{align}

Here $\theta^{(1)},\theta^{(2)},\dots,\theta^{(K)} \in \mathbb{R}^n$ are the parameters of our model. Notice that the term $\frac{1}{\sum_{j=1}^K exp(\theta^{(j)\top} \mathbf{x})}$ normalizes the distribution, so that it sums to one.

For convenience, we will also write $\theta$ to denote all the parameters of our model. When you implement softmax regression, it is usually convenient to represent $\theta$ as a $n$-by-$K$ matrix obtained by concatenating $\theta_{(1)},\theta^{(2)},\dots,\theta^{(K)}$ into columns, so that

\begin{align}
\theta = \left[\begin{array}{cccc}| & | & | & | \\
\theta^{(1)} & \theta^{(2)} & \cdots & \theta^{(K)} \\
| & | & | & |
\end{array}\right].
\end{align}


## Cost Function

We now describe the cost function that we’ll use for softmax regression. In the equation below, $1\{\cdot\}$
 is an "indicator function", such that $1\{\mathrm{a true statement}\}=1$, and $1\{\mathrm{a false statement}\}=0$. For example, $1\{2+2=4\}$ evaluates to $1$; whereas $1\{1+1=5\}$ evaluates to $0$. Our cost function will be:

\begin{align}
J(\theta) = - \left[ \sum_{i=1}^{m} \sum_{k=1}^{K}  1\left\{y_{i} = k\right\} \log \frac{\exp(\theta^{(k)\top} \mathbf{x}_i)}{\sum_{j=1}^K \exp(\theta^{(j)\top} \mathbf{x}_i)}\right]
\end{align}
 
Notice that this generalises the logistic regression cost function, which could also have been written:

\begin{align}
J(\theta) &= - \left[ \sum_{i=1}^m   (1-y^{(i)}) \log (1-h_\theta(\mathbf{x}_i)) + y^{(i)} \log h_\theta(\mathbf{x}_i) \right] \\
&= - \left[ \sum_{i=1}^{m} \sum_{k=0}^{1} 1\left\{y^{(i)} = k\right\} \log P(y^{(i)} = k | \mathbf{x}_i ; \theta) \right]
\end{align}


The softmax cost function is similar, except that we now sum over the $K$ different possible values of the class label. Note also that in softmax regression, we have that

\begin{equation}
P(y_i = k | \mathbf{x}_i ; \theta) = \frac{\exp(\theta^{(k)\top} \mathbf{x}_i)}{\sum_{j=1}^K \exp(\theta^{(j)\top} \mathbf{x}_i) }
\end{equation}

We cannot solve for the minimum of $J(\theta)$ analytically, and thus we'll resort to using gradient descent as before. Taking derivatives, one can show that the gradient is:

\begin{align}
\nabla_{\theta^{(k)}} J(\theta) = - \sum_{i=1}^{m}{ \left[ \mathbf{x}_i \left( 1\{ y_i = k\}  - P(y_i = k | \mathbf{x}_i; \theta) \right) \right]  }
\end{align}

Armed with this formula for the derivative, one can then use it directly with a gradient descent solver (or any other 1st-order gradient based optimiser).

__Use the code box below to complete the implementation of the functions that return the gradients of the softmax loss function, $\nabla_{\theta^{(k)}} J(\theta) \,\, \forall k$ and the loss function itself, $J(\theta)$:__

In [0]:
import torch
import math

# we wouldn't normally do this, but for this lab we want to work in double precision
# as we'll need the numerical accuracy later on for doing checks on our gradients:
torch.set_default_dtype(torch.float64) 

def softmax_regression_loss_grad(Theta, X, y):
    '''Implementation of the gradient of the softmax loss function.
    
    Theta is the matrix of parameters, with the parameters of the k-th class in the k-th column
    X contains the data vectors (one vector per row)
    y is a column vector of the targets
    '''
    
    A = torch.exp(X@Theta)
    den = A.sum(dim = 1, keepdim = True)
    i = torch.zeros([X.size()[0], Theta.size()[1]])
    i[torch.arange(i.size()[0]), y[:,0]] = 1
    return  -(X.t() @ (i - (A/den)))
    
    

def softmax_regression_loss(Theta, X, y):
    '''Implementation of the softmax loss function.
        
    Theta is the matrix of parameters, with the parameters of the k-th class in the k-th column
    X contains the data vectors (one vector per row)
    y is a column vector of the targets
    ''' 
    num_classes = list(Theta.size())[1]   
    num_items = list(X.size())[0]
    grad  = torch.zeros(1,1)
    
    
    for k in range(num_classes):
      for i in range(num_items):
        if y[i] == k:
          num = torch.exp(Theta[:,k:k+1].t() @ X[i:i+1,:].t())
          den = torch.zeros(1,1)
          for j in range(num_classes):
            den = den + torch.exp(Theta[:,j:j+1].t() @ X[0:1,:].t())
          g = num/den
          g = torch.log(g)
          grad = grad + g
    return -grad

__Use the following code block to confirm that your implementation is correct using gradient checking. If there are problems with your gradient or loss, go back and fix them!:__

In [270]:
 # from torch.autograd import gradcheck
from random import randrange

def grad_check(f, x, analytic_grad, num_checks=10, h=1e-3):
    sum_error = 0
    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape]) #randomly sample value to change

        oldval = x[ix].item()
        x[ix] = oldval + h # increment by h
        fxph = f(x) # evaluate f(x + h)
        x[ix] = oldval - h # increment by h
        fxmh = f(x) # evaluate f(x - h)
        x[ix] = oldval # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (abs(grad_numerical) + abs(grad_analytic) + 1e-8)
        sum_error += rel_error
        print('numerical: %f\tanalytic: %f\trelative error: %e' % (grad_numerical, grad_analytic, rel_error))
    return sum_error / num_checks

# Create some test data:
num_classes = 10
features_dim = 20
num_items = 100
Theta = torch.randn((features_dim, num_classes))
X = torch.randn((num_items,features_dim))
y = torch.torch.randint(0, num_classes, (num_items, 1))

# compute the analytic gradient
grad = softmax_regression_loss_grad(Theta, X, y)
    
# run the gradient checker    
grad_check(lambda th: softmax_regression_loss(th, X, y), Theta, grad)

numerical: -0.254098	analytic: -3.238574	relative error: 8.544963e-01
numerical: -5.175118	analytic: -0.561188	relative error: 8.043381e-01
numerical: -2.928278	analytic: -7.200081	relative error: 4.217666e-01
numerical: 4.569395	analytic: 2.629452	relative error: 2.694796e-01
numerical: -0.808050	analytic: 6.389072	relative error: 1.000000e+00
numerical: -1.828995	analytic: 1.990111	relative error: 1.000000e+00
numerical: 143.897422	analytic: 11.001000	relative error: 8.579585e-01
numerical: 1.310298	analytic: 0.511824	relative error: 4.382112e-01
numerical: 6.234292	analytic: 4.673174	relative error: 1.431239e-01
numerical: -7.709643	analytic: -6.965663	relative error: 5.069605e-02


tensor([[0.5840]])

In [271]:
Theta = torch.Tensor([[1, 0], [0, 1]])
X = torch.Tensor([[1, 0], [0, 1]])
y = torch.LongTensor([[0], [1]])
print(softmax_regression_loss(Theta, X, y))
assert torch.abs(softmax_regression_loss(Theta, X, y) - 0.6265) < 0.0001
grad = softmax_regression_loss_grad(Theta, X, y)
assert torch.torch.allclose(torch.abs(grad/0.2689), torch.ones_like(grad), atol=0.001)
print(softmax_regression_loss_grad(Theta, X, y))

tensor([[0.6265]])
tensor([[-0.2689,  0.2689],
        [ 0.2689, -0.2689]])


## Training Softmax regression with gradient descent on real data

We'll now try gradient descent with our softmax regression using the digits dataset. As before, when we looked at logistic regression, we load the data and create test and training sets. Note that this time we'll use all the classes:

In [0]:
from sklearn.datasets import load_digits

X, y = (torch.Tensor(z) for z in load_digits(10, True)) #convert to pytorch Tensors
X = torch.cat((X, torch.ones((X.shape[0], 1))), 1) # append a column of 1's to the X's
X /= 255
y = y.reshape(-1, 1) # reshape y into a column vector
y = y.type(torch.LongTensor)

# We're also going to break the data into a training set for computing the regression parameters
# and a test set to evaluate the predictive ability of those parameters
perm = torch.randperm(y.shape[0])
X_train = X[perm[0:260], :]
y_train = y[perm[0:260]]
X_test = X[perm[260:], :]
y_test = y[perm[260:]]

We now define a simple gradient descent loop to train the model:

In [273]:
alpha = 0.1
theta_gd = torch.rand((X_train.shape[1], 10))
print(theta_gd.size(), X_train.size(), y_train.size())


for e in range(0, 1000):
    gr = softmax_regression_loss_grad(theta_gd, X_train, y_train)
    theta_gd -= alpha * gr
    if e%100 == 0:
        print("Training Loss: ", softmax_regression_loss(theta_gd, X_train, y_train))

# Compute the accuracy of the test set
proba = torch.softmax(X_test @ theta_gd, 1)
print(float((proba.argmax(1)-y_test[:,0]==0).sum()) / float(proba.shape[0]))
print()


torch.Size([65, 10]) torch.Size([260, 65]) torch.Size([260, 1])
Training Loss:  tensor([[589.1023]])
Training Loss:  tensor([[248.3407]])
Training Loss:  tensor([[109.9166]])
Training Loss:  tensor([[31.6750]])
Training Loss:  tensor([[-20.8979]])
Training Loss:  tensor([[-59.6431]])
Training Loss:  tensor([[-89.7892]])
Training Loss:  tensor([[-114.0692]])
Training Loss:  tensor([[-134.0881]])
Training Loss:  tensor([[-150.8710]])
0.9290826284970722



Running the above, you should observe that the training loss decreases over time. The final accuracy on the test set is also printed and should be around 90% (it will depend on the particular training/test splits you generated as well as the initial parameters for the softmax).

# Overparameterisation in softmax regression

Softmax regression has an unusual property that it has a "redundant" set of parameters. To explain what this means, suppose we take each of our parameter vectors $\theta^{(j)}$, and subtract some fixed vector $\psi$ from it, so that every $\theta^{(j)}$ is now replaced with $\theta^{(j)}−\psi$ (for every $j=1,\dots,k$). Our hypothesis now estimates the class label probabilities as

\begin{align}
P(y^{(i)} = k | x^{(i)} ; \theta)
&= \frac{\exp((\theta^{(k)}-\psi)^\top x^{(i)})}{\sum_{j=1}^K \exp( (\theta^{(j)}-\psi)^\top x^{(i)})}  \\
&= \frac{\exp(\theta^{(k)\top} x^{(i)}) \exp(-\psi^\top x^{(i)})}{\sum_{j=1}^K \exp(\theta^{(j)\top} x^{(i)}) \exp(-\psi^\top x^{(i)})} \\
&= \frac{\exp(\theta^{(k)\top} x^{(i)})}{\sum_{j=1}^K \exp(\theta^{(j)\top} x^{(i)})}.
\end{align}

__In other words, subtracting $\psi$ from every $\theta^{(j)}$ does not affect our hypothesis’ predictions at all!__ This shows that softmax regression’s parameters are "redundant". More formally, we say that our softmax model is "overparameterised" meaning that for any hypothesis we might fit to the data, there are multiple parameter settings that give rise to exactly the same hypothesis function $h_\theta$ mapping from inputs $\mathbf{x}$ to the predictions.

Further, if the cost function $J(\theta)$ is minimized by some setting of the parameters $(\theta^{(1)},\theta^{(2)},\dots,\theta^{(k)})$, then it is also minimised by $\theta^{(1)}-\psi,\theta^{(2)}-\psi,\dots,\theta^{(k)}-\psi)$ for any value of $\psi$. Thus, the minimiser of $J(\theta)$ is not unique. 

(Interestingly, $J(\theta)$ is still convex, and thus gradient descent will not run into local optima problems. The Hessian is however singular/non-invertible, which causes a straightforward implementation of Newton's method (a second-order optimiser) to run into numerical problems.)

Notice also that by setting $\psi=\theta^{(K)}$, one can always replace $\theta^{(K)}$ with $\theta^{(K)}-\psi=\mathbf{0}$ (the vector of all $0$’s), without affecting the hypothesis. Thus, one could "eliminate" the vector of parameters $\theta^{(K)}$ (or any other $\theta^{(k)}$, for any single value of $k$), without harming the representational power of our hypothesis. Indeed, rather than optimising over the $K \cdot n$ parameters $(\theta^{(1)},\theta^{(2)},\dots,\theta^{(k)})$  (where $\theta^{(k)} \in \mathbb{R}^n$, one can instead set $\theta^{(K)}=\mathbf{0}$  and optimize only with respect to the $(K-1) \cdot n$ remaining parameters.

__Use the following block to implement the softmax gradients for the case where the final column of the parameters theta is fixed to be zero:__

In [0]:
import torch

def softmax_regression_loss_grad_0(Theta, X, y):
    '''Implementation of the gradient of the softmax loss function, with the parameters of the
    last class fixed to be zero.
    
    Theta is the matrix of parameters, with the parameters of the k-th class in the k-th column; 
            K-1 classes are included, and the parameters of the last class are implicitly zero.
    X contains the data vectors (one vector per row)
    y is a column vector of the targets
    '''
    
    # add the missing column of zeros:
    Theta = torch.cat((Theta, torch.zeros(Theta.shape[0],1)), 1)

    # YOUR CODE HERE
    
    
    # remove the last column from the gradients
    grad = grad[0:grad.shape[0], 0:grad.shape[1]-1]
    return grad

In [275]:
Theta = torch.Tensor([[1, 0], [0, 0]])
X = torch.Tensor([[1, 0], [0, 1]])
y = torch.LongTensor([[0], [1]])

grad = softmax_regression_loss_grad(Theta, X, y)
grad0 = softmax_regression_loss_grad_0(Theta[:,0:grad.shape[1]-1], X, y)
assert torch.torch.allclose(grad[:,0:grad.shape[1]-1], grad0)


UnboundLocalError: ignored

Finally, we can run gradient descent with our reduced paramter gradient function, and confirm that the results are similar to before:

In [0]:
alpha = 0.1
theta_gd = torch.rand((X_train.shape[1], 9))

for e in range(0, 1000):
    gr = softmax_regression_loss_grad_0(theta_gd, X_train, y_train)
    theta_gd -= alpha * gr

theta_gd = torch.cat((theta_gd, torch.zeros(theta_gd.shape[0], 1)), 1)
proba = torch.softmax(X_test @ theta_gd, 1)
print(float((proba.argmax(1)-y_test[:,0]==0).sum()) / float(proba.shape[0]))
print()